In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15307,2024-11-27,Brasil Nbb,19:30,Bauru,Fortaleza B.C.,1.32,2.97,150.5,1.83,1.83,-7.5,1.88,1.78,dCa1zyAe,0.757576,0.336700,0.546448,0.546448,0.094276,0.4,0.4,NaN,NaN,165.676,67.043967,0.404669,209.984,70.302606,0.334800,147.714,191.390,82.96,330.88,0.0,0.0,0.0,0.0,0.543928,0.000000,0.038640,2.05,0.410,0.780488,0.655311,0.8,0.144689,-1.64,-0.328,-6.006098,0.393446,0.3,-0.093446
15308,2024-11-27,Brasil Nbb,19:30,São José,Caxias do Sul,1.30,3.07,147.5,1.85,1.81,-8.5,1.97,1.71,UPc9YEv8,0.769231,0.325733,0.540541,0.552486,0.094964,0.2,0.2,NaN,NaN,303.600,176.421754,0.581099,314.890,183.641064,0.583191,130.164,344.850,212.94,152.49,0.0,0.0,0.0,0.0,0.572805,0.015456,0.099917,-1.99,-0.398,-0.753769,0.639098,0.4,-0.239098,1.72,0.344,6.017442,0.282271,0.3,0.017729
15309,2024-11-27,Eua Nba,00:00,Phoenix Suns,Los Angeles Lakers,1.72,2.20,233.5,1.90,1.96,-3.5,2.04,1.72,ziUh5w5k,0.581395,0.454545,0.526316,0.510204,0.035941,0.8,0.6,NaN,NaN,265.472,82.991704,0.312619,159.808,14.017550,0.087715,189.278,210.756,325.74,240.54,1.0,0.0,0.0,0.0,0.173169,0.021983,0.120359,-1.04,-0.208,-3.461538,0.618291,0.6,-0.018291,-3.78,-0.756,-1.587302,0.487095,0.4,-0.087095
15310,2024-11-27,Eua Nba,21:00,Charlotte Hornets,Miami Heat,2.41,1.61,218.5,1.93,1.93,2.5,1.99,1.82,nPxF5mqJ,0.414938,0.621118,0.518135,0.518135,0.036056,0.2,0.6,NaN,NaN,278.188,77.346038,0.278035,243.754,72.101602,0.295797,308.330,259.542,224.28,200.86,0.0,0.0,0.0,0.0,0.281436,0.000000,0.063101,1.58,0.316,4.462025,0.433840,0.6,0.166160,-0.03,-0.006,-101.666667,0.525995,0.5,-0.025995
15311,2024-11-27,Eua Nba,21:00,Cleveland Cavaliers,Atlanta Hawks,1.15,6.04,237.5,1.81,1.99,-12.5,2.00,1.81,EVU0TjyQ,0.869565,0.165563,0.552486,0.502513,0.035128,0.6,0.4,NaN,NaN,176.528,72.342028,0.409805,416.220,358.797764,0.862039,154.406,450.156,139.08,360.84,0.0,0.0,0.0,0.0,0.961823,0.066989,0.070525,1.32,0.264,0.568182,0.755255,0.8,0.044745,9.12,1.824,2.763158,0.403726,0.3,-0.103726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15424,2024-11-27,Eua Ncaa,22:30,Xavier,Michigan,2.10,1.73,151.5,1.80,1.91,1.5,1.91,1.80,tbSVFZcS,0.476190,0.578035,0.555556,0.523560,0.054225,0.6,0.2,NaN,NaN,118.014,27.279015,0.231151,243.864,265.564407,1.088986,119.094,309.526,78.78,87.00,0.0,0.0,0.0,0.0,0.136621,0.041931,0.041931,0.43,0.086,12.790698,0.641153,0.6,-0.041153,-3.84,-0.768,-0.950521,0.382336,0.2,-0.182336
15425,2024-11-27,Eua Ncaa,23:00,Marquette,Stonehill,1.91,1.91,138.5,1.83,1.83,-34.5,1.91,1.91,Yuqs1jyF,0.523560,0.523560,0.546448,0.546448,0.047120,0.2,0.4,NaN,NaN,110.984,14.743510,0.132844,463.342,596.169788,1.286673,110.986,503.592,124.64,238.17,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.11,0.422,2.156398,0.774075,0.9,0.125925,-5.00,-1.000,-0.910000,0.267512,0.0,-0.267512
15426,2024-11-27,Eua Ncaa,23:00,Montana,CS Northridge,1.83,2.00,152.5,1.80,1.95,-2.5,2.00,1.71,v5j5ySUb,0.546448,0.500000,0.555556,0.512821,0.046448,0.6,0.4,NaN,NaN,190.958,202.227286,1.059014,173.046,55.511591,0.320791,103.432,206.346,81.42,184.30,0.0,0.0,0.0,0.0,0.062772,0.056569,0.110545,0.01,0.002,415.000000,0.781759,0.8,0.018241,1.55,0.310,3.225806,0.347948,0.5,0.152052
15427,2024-11-27,Europa Euroliga Feminina,13:00,Fenerbahce F,Polkowice F,1.01,15.00,154.5,1.90,1.88,-21.5,1.94,1.77,jmbXO24l,0.990099,0.06

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Indiana Pacers,Portland Trail Blazers,226.5,1.81,0.5597,Over
1,22:00,Eua Nba,Memphis Grizzlies,Detroit Pistons,227.5,1.82,0.7905,Over
2,22:00,Eua Nba,New Orleans Pelicans,Toronto Raptors,222.5,1.82,0.7766,Over
3,21:00,Eua Nba,Philadelphia 76ers,Houston Rockets,214.5,1.83,0.9301,Over
4,23:00,Eua Nba,Phoenix Suns,Brooklyn Nets,220.5,1.85,0.9041,Over
5,21:00,Eua Ncaa,NC Wilmington,Colgate,145.5,1.87,0.8302,Over
6,22:00,Eua Ncaa,St. Louis Billikens,UMass Lowell,164.5,1.91,0.8342,Over
7,23:00,Eua Ncaa,Marquette,Stonehill,138.5,1.83,0.6585,Over
